In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import pickle
from sklearn.model_selection import train_test_split

In [2]:
# Load already augmented data from saved pickle files
X_train_aug = pickle.load(open("../../pickles/X_train_aug.pkl", "rb"))
y_train_aug = pickle.load(open("../../pickles/y_train_aug.pkl", "rb"))
X_test = pickle.load(open("../../pickles/X_test.pkl", "rb"))
y_test = pickle.load(open("../../pickles/y_test.pkl", "rb"))

In [ ]:
# Confirm the shapes of the data
display(X_train_aug.shape, y_train_aug.shape, X_test.shape, y_test.shape)

In [4]:
# further split the training data into training and validation
X_train, X_val, y_train, y_val = train_test_split(X_train_aug, y_train_aug, test_size=0.2, random_state=42)

In [ ]:
# Create the model using Convolutional Neural Network
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu'),   # 32 filters, 3x3 kernel
    layers.MaxPooling2D((2, 2)),                    # 2x2 pool size
    layers.Conv2D(64, (3, 3), activation='relu'),   # 64 filters, 3x3 kernel
    layers.MaxPooling2D((2, 2)),                    # 2x2 pool size
    layers.Conv2D(64, (3, 3), activation='relu'),   # 64 filters, 3x3 kernel       
    layers.Flatten(),                               # Flatten the 3D output to 1D
    layers.Dense(64, activation='relu'),            # 64 neurons
    layers.Dense(6, activation='softmax')           # 6 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
history = model.fit(
    X_train, y_train, 
    batch_size=batch_size, 
    epochs=epochs, 
    validation_data=(X_val, y_val))

In [ ]:
# Evaluate the model
model.evaluate(X_test, y_test)

In [ ]:
model.save("../../models/model_sigmoid_ohe_10e.h5")